In [1]:

from src.Locking.LL import LogicLocking
import src.utils as utils
from src.Netlist.AST import AST

# "/home/alira/FYP_FINAL/input_files/ASSURE_LOCKED/design4/design4_netlist.v"

top="tate_pairing_0_obf"

locked = AST(file_path=f"./output_files/ASSURE_VERILOG/{top}org.json",rw='r',filename=f"{top}locked_test") #Run to read in AST Format

extract_modules_def took 0.0032684803009033203 seconds to complete its execution.
Reading LL file
	 Loading json file
	 Done Loading json file
	 Loading Top module in AST
	 Done Loading Top module tate_pairing_0_obf in AST
	 Loading module data in AST
		 Loading module tate_pairing_0_obf in AST
		 Done Loading module tate_pairing_0_obf in AST
	 Done Loading module data in AST
	 Loading gate level data in AST
	 Done Loading gate level data in AST
Done Reading LL file


In [2]:
locked.top_module.save_graph()

In [19]:
import os
import re
import shutil
import src.utils as utils
import src.Parser.verilog_parser as verilog_parser
import src.Parser.bench_parser as bench_parser
import src.Parser.conv as conv
from src.Netlist.AST import AST,module
import src.Attacks.SATAttack.SATAttack as satattack
import src.Attacks.SATAttack.benchmarks_custom as benchmarks
import src.Netlist.netlist as netlist

In [20]:
# locked_filename="input_files/benchmark_bench/rnd/apex2_enc05.bench"
# unlocked_filename="input_files/benchmark_bench/original/apex2.bench"
# ASSURE_BENCH
# top="apex2_enc05"
# top="apex2"

circuit_name="usb_phy"

top=circuit_name+"_0_obf"
locked = AST(file_path=f"output_files/ASSURE_BENCH/{top}org.json",rw='r',filename=f"{top}locked_test") #Run to read in AST Format
top=circuit_name
unlocked = AST(file_path=f"output_files/ASSURE_BENCH/{top}org.json",rw='r',filename=f"{top}locked_test") #Run to read in AST Format



extract_modules_def took 0.0003352165222167969 seconds to complete its execution.
Reading LL file
	 Loading json file
	 Done Loading json file
	 Loading Top module in AST
	 Done Loading Top module usb_phy_0_obf in AST
	 Loading module data in AST
		 Loading module usb_phy_0_obf in AST
		 Done Loading module usb_phy_0_obf in AST
	 Done Loading module data in AST
	 Loading gate level data in AST
	 Done Loading gate level data in AST
Done Reading LL file
extract_modules_def took 0.0002808570861816406 seconds to complete its execution.
Reading LL file
	 Loading json file
	 Done Loading json file
	 Loading Top module in AST
	 Done Loading Top module usb_phy in AST
	 Loading module data in AST
		 Loading module usb_phy in AST
		 Done Loading module usb_phy in AST
	 Done Loading module data in AST
	 Loading gate level data in AST
	 Done Loading gate level data in AST
Done Reading LL file


In [21]:
import networkx as nx

tmpgraph=nx.DiGraph()
for init,val in locked.top_module.FF_tech['DFFQ_X1'].copy().items():
  locked.top_module.FF_tech['DFFQ_X1'].pop(init)
  Q=val["outputs"]
  D=val["inputs"]

  tmpgraph.add_node(init,type="init")
  tmpgraph.add_node(Q,type="Q")

  
  tmpgraph.add_node(D,type="D")

  tmpgraph.add_edge(D, init)
  tmpgraph.add_edge(init, Q)

In [22]:
tmpnodes=[node for node, in_degree in tmpgraph.in_degree() if in_degree == 0]

In [23]:
for driver in tmpnodes:
  # print(tmpgraph.nodes[i]["type"])
  assigned=[]
  curr=driver
  while(1):
    succ=list(tmpgraph.successors(curr))
    if(succ==[]):
      break
    else:
      if(tmpgraph.nodes[curr]["type"]=="init"):
        assigned.append(succ[0])
      curr=succ[0]

  if(driver in unlocked.top_module.io["inputs"]):
    print("HERE1 ",driver)
  
  if(driver in unlocked.top_module.io["outputs"]):
    print("HERE2 ",driver)
  
  if(assigned[-1] in unlocked.top_module.io["outputs"]):
    print("HERE3 ",driver,assigned[-1])
  
  # for k in assigned:
  #   print({'inputs': [driver], 'outputs': k})
  # print()
  # break

HERE1  rxdn
HERE3  rxdn LineState_o_1_
HERE3  i_rx_phy_rxdp_s0 LineState_o_0_
HERE1  rxd
HERE3  N24 usb_rst
HERE3  n2144 RxValid_o
HERE3  n1017 txdp
HERE3  n1016 txdn
HERE3  i_tx_phy_N49 TxReady_o
HERE3  n1039 DataIn_o_6_
HERE3  n1038 DataIn_o_5_
HERE3  n1037 DataIn_o_4_
HERE3  n1036 DataIn_o_3_
HERE3  n1035 DataIn_o_2_
HERE3  n1034 DataIn_o_1_
HERE3  n1033 DataIn_o_0_
HERE3  n1040 DataIn_o_7_


In [24]:
driver,assigned

('n1013', ['i_tx_phy_state_1'])

In [25]:
import random
key = random.choice(list(unlocked.top_module.gates["BUF"].keys()))
key

'BUF_X12'

In [26]:
# unlocked.top_module.gates["BUF"][key][init]={'inputs': [D], 'outputs': Q}

In [27]:
for i,j in zip([node for node, out_degree in tmpgraph.out_degree() if out_degree == 0],[node for node, in_degree in tmpgraph.in_degree() if in_degree == 0]):
  print(f"{i:<30} {j:<30}")

LineState_o_1_                 rxdn                          
i_rx_phy_rxdn_s                i_rx_phy_N128                 
LineState_o_0_                 i_rx_phy_rxdp_s0              
i_rx_phy_rxdp_s                i_rx_phy_N116                 
i_rx_phy_rxd_s1                rxd                           
i_rx_phy_rxd_s                 n1051                         
i_rx_phy_rxd_r                 n2508                         
i_rx_phy_rx_en                 N24                           
fs_ce                          n1011                         
usb_rst                        n1027                         
i_tx_phy_tx_ip_sync            n1001                         
i_tx_phy_txoe_r1               i_tx_phy_N143                 
i_tx_phy_one_cnt_1             n1023                         
i_tx_phy_sft_done_r            n1014                         
i_tx_phy_data_done             n1015                         
i_tx_phy_state_0               n1020                         
i_tx_phy

In [28]:
utils.save_graph(tmpgraph,svg=True,file_name="tmp_tmpgraph")

In [32]:
FF_inits=[]
FF_inputs=[]
FF_outputs=[]

for i,j in locked.top_module.FF_tech['DFFQ_X1'].items():
  # print(i,j["outputs"],j["inputs"])
  FF_inits.append(i)
  FF_inputs.append(j["inputs"])
  FF_outputs.append(j["outputs"])


In [30]:
len(utils.get_common_elements(FF_inputs,FF_outputs))

0

In [ ]:
common=[]
for init,inp,out in zip(FF_inits,FF_inputs,FF_outputs):
  if(inp in FF_outputs and inp not in common):
    # print("INP ",init,inp,out)
    common.append(inp)
  if(out in FF_inputs and out not in common):
    # print("OUT ",init,inp,out)
    common.append(out)

In [ ]:
common

In [ ]:
for i,j in tmp.copy().items():
  
  if(j[1] in tmp):
    print(i,tmp[i])

In [ ]:
import random
def FF_Removal(obj):
  tmp={j["outputs"]:(i,j["inputs"]) for i,j in obj.top_module.FF_tech['DFFQ_X1'].items()}
  tmpr={i:tmp.pop(i) for i,j in tmp.copy().items() if(j[1] in tmp)}
  obj.top_module.FF_tech.pop('DFFQ_X1')

  key = random.choice(list(obj.top_module.gates["BUF"].keys()))
  for Q,(init,D) in tmp.items():
    # Input Latched Flip-Flops
    if(D in obj.top_module.io["inputs"]):
      # print("I",init)
      obj.top_module.gates["BUF"][key][init]={'inputs': [D], 'outputs': Q}
    # Output Latched Flip-Flops
    elif(Q in obj.top_module.io["outputs"]):
      obj.top_module.gates["BUF"][key][init]={'inputs': [D], 'outputs': Q}
    # Output Latched Flip-Flops
    elif(D in obj.top_module.io["outputs"]):
      obj.top_module.gates["BUF"][key][init]={'inputs': [D], 'outputs': Q}
    else:
      obj.top_module.io["inputs"][Q]=obj.top_module.io["wires"].pop(Q)
      obj.top_module.io["input_ports"]+=f"{Q},"

      obj.top_module.io["outputs"][D]=obj.top_module.io["wires"].pop(D)
      obj.top_module.io["output_ports"]+=f"{D},"

      # print("Inter",D,Q) 
      # print(init,D,Q)
      # print(f"{D}  {Q}")
  obj.top_module.gen_graph()
  return tmpr,tmp

In [ ]:
tmpr,tmp=FF_Removal(locked)
# FF_Removal(unlocked)

In [ ]:
for Q,(init,D) in tmpr.items():
  # print(f"{init:<50} {D:<50} {Q}")
  if(D in tmp):
    print(f"{D:<30} {Q:<30} {tmp[D][1]}")
  else:
    print(f"{D:<30} {Q}")
  # for Q2,(init2,D2) in tmpr.items():
  #   if(Q==D2):
  #     print(Q,Q2)
  #   if(D==Q2):
  #     print(Q,Q2)


In [ ]:
locked.top_module.nodeio("i_tx_phy_hold_reg_1")
print()
locked.top_module.nodeio("DFF_out_i_tx_phy_hold_reg_d_1_")

In [ ]:
locked.top_module.io["Clock_pins"]

In [ ]:
locked.top_module.gen_graph()

In [ ]:
analyze=netlist.Analyze_Netlist(locked.top_module,unlocked.top_module)

In [ ]:
for i,j in zip(locked.top_module.io['outputs'],unlocked.top_module.io['outputs']):
    print(i,j)

In [ ]:
keys={}
for ii in analyze.attack_outputs:
  print("START SAT ATTACK")
  satobj=analyze.attack_outputs[ii]
  satattackobj=satattack.SatAttack(file_type="obj",satobj=satobj)
  # satobj=satattack.SatAttack(file_type="b",locked_filename=locked_filename, unlocked_filename=unlocked_filename)
  print(satattackobj.run())
  print(satattackobj.iterations)
  keys[ii]=satattackobj.key
  # break

In [ ]:
final_key={}
for o,k in keys.items():
  for i in k:
    # print(i)
    if(i not in final_key):
      final_key[i]=keys[o][i]
      # print(i,keys[o][i])
    else:
      print(i,keys[o][i],final_key[i])

In [ ]:
final_key

In [ ]:
locked.top_module.io["outputs"],unlocked.top_module.io["outputs"]

In [ ]:
satattackobj.key

In [ ]:
satattackobj._key_string(satattackobj.key)

In [ ]:
satattackobj.key

In [ ]:
# AND = 1/4
# OR = 3/4
# NOT = 1/2
# XOR = 1/2
# NAND = 3/4
# NOR = 1/4
# XNOR = 1/2

# AND gate:
# sAND = 0.5*(s1 + s2) + s1*s2 - 0.25

# OR gate:
# sOR = 0.5*(s1 + s2) + s1*s2 + 0.25

# NOT gate:
# sNOT = -s1

# XOR gate:
# sXOR = 0.5*(s1 + s2) - s1*s2

# NAND gate:
# sNAND = -0.5*(s1 + s2) - s1*s2 + 0.25

# NOR gate:
# sNOR = -0.5*(s1 + s2) - s1*s2 - 0.25

# XNOR gate:
# sXNOR = 0.5*(s1 + s2) + s1*s2

In [ ]:
# import pickle
# pickle.dump(std1, file)
# with open("tmp.v","w") as f:
#     f.write(verilog_init)

In [ ]:
# # path="/mnt/d/alis_files/LAPTOP/alis_files/university_files/PROJECTS_2022-2023/FYP/linux/65GP_STDCELL_verilog/tcbn65gplus.v"
# # path= "vlib/mycells.v"
# # path="input_files/warm-up_obfuscation/internal_lib.v"
# path="/home/alira/FYP_FINAL/input_files/ASSURE_LOCKED/modulefiles.v"
# # path="/mnt/d/alis_files/LAPTOP/alis_files/university_files/PROJECTS_2022-2023/FYP/linux/gnnre/Netlist_to_graph/Circuits_datasets/Interconnected-Modules/Train_add_mul_sub_64_bit_Syn_65nm.v"
# with open(path,"r") as f:
#   module_txt=f.read()
# gate_mapping_vlib,gates_vlib,FF_vlib=verilog_parser.extract_modules_def(module_txt)
# vlib_var=gate_mapping_vlib,gates_vlib,FF_vlib
# bench=open("input_files/ASSURE_LOCKED/design4/design4.bench").read()

In [ ]:
# gate_mapping_vlib,gates_vlib,FF_vlib=locked.gate_mapping_vlib,locked.gates_vlib,locked.FF_vlib

In [ ]:
# locked.top_module.gates.keys(),locked.top_module.FF_tech.keys()

In [ ]:
# tech_map={i:i for i in list(locked.top_module.gates,locked.top_module.FF_tech)}

In [ ]:
# for gate, gate_list in gate_mapping_vlib.items():
#   print(gate, gate_list)

In [ ]:
# path="/mnt/d/alis_files/LAPTOP/alis_files/university_files/PROJECTS_2022-2023/FYP/linux/65GP_STDCELL_verilog/tcbn65gplus.v"
# path= "vlib/mycells.v"
# path="input_files/warm-up_obfuscation/internal_lib.v"
path="/home/alira/FYP_FINAL/input_files/ASSURE_LOCKED/modulefiles.v"
# path="/mnt/d/alis_files/LAPTOP/alis_files/university_files/PROJECTS_2022-2023/FYP/linux/gnnre/Netlist_to_graph/Circuits_datasets/Interconnected-Modules/Train_add_mul_sub_64_bit_Syn_65nm.v"
with open(path,"r") as f:
  module_txt=f.read()
gate_mapping_vlib2,gates_vlib2,FF_vlib2=verilog_parser.extract_modules_def(module_txt)

In [ ]:
gate_mapping_vlib,gates_vlib,FF_vlib=locked.gate_mapping_vlib,locked.gates_vlib,locked.FF_vlib

In [ ]:
tech_map=verilog_parser.gen_tech_map(gate_mapping_vlib,FF_vlib,gates_vlib,gate_mapping_vlib2,FF_vlib2,gates_vlib2)

In [ ]:
txt=verilog_parser.gates_to_txt_techmap(locked.top_module.gates,gates_vlib2,tech_map)
txt+=verilog_parser.FF_to_txt_techmap(locked.top_module.FF_tech,FF_vlib2,tech_map)

In [ ]:
with open("tmp.v","w") as f:
    f.write(txt)

In [ ]:
# path_dir=r"/mnt/d/alis_files/LAPTOP/alis_files/university_files/GDRIVE/Hardware security FYP/gnnre/Netlist_to_graph/Circuits_datasets/Interconnected-Modules"
# paths=[os.path.join(path_dir,i) for i in os.listdir(path_dir) if(i[:5]=="Train")]

# import re
# def analyze_init(paths):
#   items={}
#   for path in paths:
#     with open(path,"r") as f:
#       module_txt=f.read()
    
#     for i in re.findall(r"(\w+) (\w+) \((.*)\);",module_txt)[1:]:
#       tmpi=re.findall(r'\.(\S+)\( ?[^\(\),]+\)',i[2])
#       port="("
#       for k in tmpi[:-1]:
#         port+=f".{k}({{{k}}}), "
#       port+=f".{tmpi[-1]}({{{tmpi[-1]}}})"+");"
#       if(i[0] not in items):
#         items[i[0]]=[port]
#       elif(items[i[0]]!=[port]):
#         raise Exception("Items have mismatching initiation")
#   return items

In [ ]:
# items=analyze_init(paths)

In [ ]:

# with open("input_files/ASSURE_LOCKED/modulefiles.v","r") as f:
#   module_txt=f.read()


# with open("tmp/tmpmod.v","w") as f:
#   f.write(module_txt)


# # with open("tmp.v","w") as f:
# #   f.write(t)

# gate_mapping,gates,FF=verilog_parser.extract_modules_def(module_txt)

# path="input_files/ASSURE_LOCKED/design4/design4_netlist.v"
# # "input_files/test.v"
# # "input_files/ASSURE_LOCKED/design4/design4_netlist.v"




# # # verilog=tmp1()
# # verilog=tmp2()
# # with open("tmp/tmp.v","w") as f:
# #   f.write(verilog)
# # # print(verilog)

# # print("HERE")


In [ ]:
# gate_tech,sub_module,(FF_tech,Clock_pins,Reset_pins)=verilog_parser.gates_module_extraction(verilog,gate_mapping,gates,FF)

In [ ]:
# return_top =lambda i:re.findall(r'(module\s+(\w+)\s*\(.*?\)\s*;.*?endmodule)', utils.format_verilog_org(open(i).read()), re.DOTALL)[0][1]
# orgfile_names={i:[return_top(f"input_files/ASSURE_LOCKED/{i}/{k}") for k in os.listdir("input_files/ASSURE_LOCKED/"+i) if("oracle" in k and ".v" in k)][0] for i in os.listdir("input_files/ASSURE_LOCKED") if("design" in i)}

# path="input_files/ASSURE_LOCKED/ASSURE_bench_to_verilog/"
# files=[[os.path.join(path,i,j) for j in os.listdir(path+i)] for i in os.listdir(path)]
# files.sort()
# for i in files:
#   i.sort()

# files=[([f"input_files/ASSURE_LOCKED/{i}/{k}" for k in os.listdir("input_files/ASSURE_LOCKED/"+i) if(".bench" in k and "design" in k)][0],[f"input_files/ASSURE_LOCKED/{i}/{j}" for j in os.listdir("input_files/ASSURE_LOCKED/"+i) if(".bench" in j and "oracle" in j)][0]) for i in os.listdir("input_files/ASSURE_LOCKED") if("design" in i)]
# files.sort()
# outpath="input_files/ASSURE_LOCKED/ASSURE_bench_to_verilog/"



# for i in files[4:5]:
#   design=i[0]
#   oracle=i[1]
#   top=design.split("/")[-1].split(".bench")[0]
#   top_name=orgfile_names[top]
#   print(top,top_name)

#   txt_design=open(design).read()
#   txt_oracle=open(oracle).read()


  
#   if not os.path.isdir(outpath+top):
#     os.mkdir(outpath+top)

#   print(top+" oracle")
#   verilog_oracle=bench_to_verilog_vlib(txt_oracle,vlib_var,modulename=top_name)
  
#   with open(outpath+f"{top}/{top}_oracle.v","w") as f:
#     f.write(verilog_oracle)

#   print(top+" design")
  
#   verilog_design=bench_to_verilog_vlib(txt_design,vlib_var,modulename=top_name+"_0_obf")
  
#   with open(outpath+f"{top}/{top}_design.v","w") as f:
#     f.write(verilog_design)


# for i in files:
#   design=i[0]
#   oracle=i[1]
#   top=design.split("/")[-2]
#   top_design= orgfile_names[top]+"_0_obf"
#   top_oracle=orgfile_names[top]
#   print(top,top_design,top_oracle)
  
#   print(design,oracle,top_oracle,top_design)
  
#   print("DOING ",top_oracle)
#   obj=AST(file_path=oracle,rw="w",flag="v",top=top_oracle,filename=f"{top_oracle}org",output_dir_path="output_files/ASSURE_BENCH",vlibpath="input_files/ASSURE_LOCKED/modulefiles.v",synth=False)#Run to Read in Verilog Design
#   print("DOING ",top_design)
#   obj=AST(file_path=design,rw="w",flag="v",top=top_design,filename=f"{top_design}org",output_dir_path="output_files/ASSURE_BENCH",vlibpath="input_files/ASSURE_LOCKED/modulefiles.v",synth=False,locked=True)#Run to Read in Verilog Design
#   break

  

In [ ]:
# json_files=["./output_files/"+i for i in os.listdir("./output_files/") if os.path.isfile("./output_files/"+i)]
# orgfile_names=[i.split(".bench")[0] for i in os.listdir("input_files/benchmark_bench/original")]
# json_files.sort()

# return_top =lambda i:re.findall(r'(module\s+(\w+)\s*\(.*?\)\s*;.*?endmodule)', utils.format_verilog_org(open(i).read()), re.DOTALL)[0][1]

# orgfile_names=[[return_top(f"input_files/ASSURE_LOCKED/{i}/{k}") for k in os.listdir("input_files/ASSURE_LOCKED/"+i) if("oracle" in k and ".v" in k)][0] for i in os.listdir("input_files/ASSURE_LOCKED") if("design" in i)]

# for i in json_files:
#   for j in orgfile_names:
#     if(j in i.split("/")[-1].split("org")[0]):
#       print(i,"output_files/ASSURE_VERILOG/"+i.split("/")[-1])
#       shutil.move(i,"output_files/ASSURE_VERILOG/"+i.split("/")[-1])

In [ ]:
# files=[([f"input_files/ASSURE_LOCKED/{i}/{k}" for k in os.listdir("input_files/ASSURE_LOCKED/"+i) if(".v" in k and "design" in k)][0],[f"input_files/ASSURE_LOCKED/{i}/{j}" for j in os.listdir("input_files/ASSURE_LOCKED/"+i) if(".v" in j and "oracle" in j)][0]) for i in os.listdir("input_files/ASSURE_LOCKED") if("design" in i)]
# files.sort()

In [ ]:
# import re
# import os
# files=[([f"input_files/ASSURE_LOCKED/{i}/{k}" for k in os.listdir("input_files/ASSURE_LOCKED/"+i) if(".v" in k and "design" in k)][0],[f"input_files/ASSURE_LOCKED/{i}/{j}" for j in os.listdir("input_files/ASSURE_LOCKED/"+i) if(".v" in j and "oracle" in j)][0]) for i in os.listdir("input_files/ASSURE_LOCKED") if("design" in i)]
# files.sort()
# notdone=[]
# for i in files[1:]:
#   design=i[0]
#   oracle=i[1]

#   tmp=open(design).read()
#   top_design = re.findall(r'(module\s+(\w+)\s*\(.*?\)\s*;.*?endmodule)', utils.format_verilog_org(tmp), re.DOTALL)[0][1]
#   tmp=open(oracle).read()
#   top_oracle = re.findall(r'(module\s+(\w+)\s*\(.*?\)\s*;.*?endmodule)', utils.format_verilog_org(tmp), re.DOTALL)[0][1]
#   # try:
#   #   print("DONE ",top_oracle)
#   obj=AST(file_path=oracle,rw="w",flag="v",top=top_oracle,filename=f"{top_oracle}org",vlibpath="input_files/ASSURE_LOCKED/modulefiles.v",synth=False)#Run to Read in Verilog Design
#   # except:
#   #   print("FAILED ",top_oracle)
#   #   notdone.append(oracle)
#   # try:
#   print("DONE ",top_design)
#   obj=AST(file_path=design,rw="w",flag="v",top=top_design,filename=f"{top_design}org",vlibpath="input_files/ASSURE_LOCKED/modulefiles.v",synth=False,locked=True)#Run to Read in Verilog Design
#   # except:
#   #   print("FAILED ",top_design)
#   #   notdone.append(design)
#   break

In [ ]:
# notdone=[]
# for i in os.listdir("input_files/benchmark_bench/original"):
#   top=i.split(".bench")[0]
#   unlocked_filename=os.path.join("input_files/benchmark_bench/original",i)
#   try:
#     obj=AST(file_path=unlocked_filename,rw="w",flag="b",top=top,filename=f"{top}org",vlibpath="input_files/ASSURE_LOCKED/modulefiles.v",synth=False)#Run to Read in Verilog Design
#   except:
#     notdone.append(unlocked_filename)
#   for j in os.listdir("input_files/benchmark_bench/rnd"):
#     if(i.split(".bench")[0] in j):
#       top=j.split(".bench")[0]
#       locked_filename=os.path.join("input_files/benchmark_bench/rnd",j)
#       print(j.split(".bench")[0],locked_filename)
#       try:
#         obj=AST(file_path=locked_filename,rw="w",flag="b",top=top,filename=f"{top}org",vlibpath="input_files/ASSURE_LOCKED/modulefiles.v",synth=False,locked=True)#Run to Read in Verilog Design
#       except:
#         notdone.append(locked_filename)